In [1]:
print("dsadsadsa")

dsadsadsa


In [28]:
import pandas as pd
from sklearn.utils import resample
from pathlib import Path

dataset_dir = Path("./dataset")
dataset_dir.mkdir(parents=True, exist_ok=True)  # Ensure the directory exists
dataset_path = dataset_dir / "creditcard-data.csv"
df = pd.read_csv(dataset_path)
df.head

<bound method NDFrame.head of          Time        V1        V2        V3        V4        V5        V6  \
0     82450.0  1.314539  0.590643 -0.666593  0.716564  0.301978 -1.125467   
1     50554.0 -0.798672  1.185093  0.904547  0.694584  0.219041 -0.319295   
2     55125.0 -0.391128 -0.245540  1.122074 -1.308725 -0.639891  0.008678   
3    116572.0 -0.060302  1.065093 -0.987421 -0.029567  0.176376 -1.348539   
4     90434.0  1.848433  0.373364  0.269272  3.866438  0.088062  0.970447   
..        ...       ...       ...       ...       ...       ...       ...   
979  169142.0 -1.927883  1.125653 -4.518331  1.749293 -1.566487 -2.010494   
980  169347.0  1.378559  1.289381 -5.004247  1.411850  0.442581 -1.326536   
981  169351.0 -0.676143  1.126366 -2.213700  0.468308 -1.120541 -0.003346   
982  169966.0 -3.113832  0.585864 -5.399730  1.817092 -0.840618 -2.943548   
983  170348.0  1.991976  0.158476 -2.583441  0.408670  1.151147 -0.096695   

           V7        V8        V9  ...       

In [29]:
# Load the dataset (Ensure you have 'creditcard.csv' in your working directory)
df = pd.read_csv(dataset_path)

# Check class distribution
print("Original class distribution:")
print(df["Class"].value_counts())

# Separate majority and minority classes
df_majority = df[df["Class"] == 0]  # Non-fraudulent transactions
df_minority = df[df["Class"] == 1]  # Fraudulent transactions

# Apply undersampling: Reduce majority class to match minority class size
df_majority_undersampled = resample(df_majority, 
                                    replace=False,    # Sample without replacement
                                    n_samples=len(df_minority),  # Match minority class size
                                    random_state=42)  # Reproducibility

# Combine minority class with undersampled majority class
df_undersampled = pd.concat([df_majority_undersampled, df_minority])

# Check new class distribution
print("\nNew class distribution after undersampling:")
print(df_undersampled["Class"].value_counts())

# Save the new dataset
#new_dataset_path = dataset_dir / "creditcard_undersampled.csv"
df_undersampled.to_csv(dataset_path, index=False)
print("\nUndersampled dataset saved as 'creditcard_undersampled.csv'.")

Original class distribution:
Class
0    492
1    492
Name: count, dtype: int64

New class distribution after undersampling:
Class
0    492
1    492
Name: count, dtype: int64

Undersampled dataset saved as 'creditcard_undersampled.csv'.


In [30]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
X = df_undersampled.drop(columns=["Class"])
y = df_undersampled["Class"]

#Normalize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
    
# Split into train, validation, and test sets
X_train, X_temp, y_train, y_temp = train_test_split(X_scaled,y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)
    
# Convert to DataFrame for saving
train_df = pd.DataFrame(X_train, columns=X.columns)
train_df["Class"] = y_train
val_df = pd.DataFrame(X_val, columns=X.columns)
val_df["Class"] = y_val
test_df = pd.DataFrame(X_test, columns=X.columns)
test_df["Class"] = y_test

#logger.info("Preprocessed data and split into train, validation, and test sets.")



In [20]:
import boto3

# MinIO credentials and endpoint
endpoint_url = "http://localhost:9000"  # Adjust if running remotely
aws_access_key_id = "user"
aws_secret_access_key = "WW4e69Wwcv0w"
bucket_name = "models-bucket"
file_path = "userprefix.json"
object_name = "uploaded_example.txt"

# Create S3 client
s3_client = boto3.client(
    "s3",
    endpoint_url=endpoint_url,  # MinIO URL
    aws_access_key_id=aws_access_key_id,
    aws_secret_access_key=aws_secret_access_key
)

# Upload file
s3_client.upload_file(file_path, bucket_name, object_name)

print(f"File {file_path} uploaded to {bucket_name} as {object_name}")

File userprefix.json uploaded to models-bucket as uploaded_example.txt
